### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
34,42970679,42244,79085,3,0,0,1150,6,2025-07-25 19:24,2025-07-25 19:46,3,1,5601,6,1
35,42974936,28732,79085,51530,0,0,1151,0,2025-07-26 19:30,2025-07-26 19:30,0,1,5931,4,1
36,42740342,79085,78686,1349,11,71,1151,1,2025-07-27 21:59,2025-07-27 21:59,15,0,5846,2,1
37,42977531,94649,79085,51530,0,0,1151,1,2025-08-03 19:35,2025-08-03 19:35,2,0,8256,5,1
38,42974933,90579,79085,51530,0,0,1151,2,2025-07-28 18:00,2025-07-28 18:00,0,1,2932,3,1
39,42977498,144925,79085,51530,0,0,1151,2,2025-08-04 18:20,2025-08-04 18:20,0,1,2713,4,1
40,42974934,90579,79085,51530,0,0,1151,3,2025-07-29 18:00,2025-07-29 18:00,0,1,1848,4,1
41,42977502,88881,79085,51530,0,0,1151,3,2025-08-05 22:00,2025-08-05 22:00,2,1,2598,2,1
42,42740343,78686,79085,1349,12,71,1151,4,2025-08-06 21:59,2025-08-06 21:59,0,11,10109,3,1
43,42975748,23377,79085,51530,0,0,1151,5,2025-08-07 19:30,2025-08-07 19:30,2,0,6678,4,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

10

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,42244,18767,35639,1,27,0,0,0,rombo meva,10,7,8,42970679
1,79085,51300,34428,1,1,0,0,0,5-4-1a,6,7,8,42970679
0,28732,31394,33826,0,14,6,1,0,dzeko,11,9,9,42974936
1,79085,38679,36247,3,10,4,1,0,4-3-3aRU,9,9,11,42974936
0,79085,13849,42526,1,30,1,0,0,3-4-3BC,10,10,8,42740342
1,78686,56208,27531,0,0,2,1,0,Ptolomeu,6,4,4,42740342
0,94649,26032,46979,1,8,4,1,1,lipka2,9,12,14,42977531
1,79085,43927,22980,0,1,3,1,2,5-4-1a,8,10,11,42977531
0,90579,26776,29696,1,1,2,0,0,DEFENSIVA ALA E,9,10,11,42974933
1,79085,43177,40257,1,4,0,0,0,5-4-1a,9,9,12,42974933


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1900

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,23775,12,31,10,1,1,35,11,31524035010011,1349
1,122169,12,30,10,0,2,68,14,30554068010008,1349
2,6276,12,29,9,2,1,38,6,29532038009012,1349
3,124139,12,23,7,2,3,35,13,23522035007010,1349
4,78751,12,20,6,2,4,19,14,20505019006002,1349
5,79587,12,19,6,1,5,29,17,19512029006001,1349
6,23755,12,17,5,2,5,24,28,17496024005009,1349
7,78671,12,15,5,0,7,25,26,15499025005007,1349
8,79085,12,9,2,3,7,28,18,9510028002006,1349
9,79866,12,8,2,2,8,20,22,8498020002005,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()